In [0]:
import numpy as np
from scipy.optimize import linprog

def nn2na(NN):
    # Get every location where exist an arc:
    idxs = np.argwhere(NN)
    # Preallocate NA matrix, dimension is (nodes, arcs)
    NA = np.zeros([NN.shape[0], idxs.shape[0]]).astype(int)
    C = np.zeros(NA.shape[1])
    # Loop in every arc, complete from (1) to (-1)
    for i, arc in enumerate(idxs):
        # Node arc:
        NA[arc[0], i] = 1 # From
        NA[arc[1], i] = -1 # To

    arc_idxs = [(arc[0], arc[1]) for arc in idxs]

    return NA, arc_idxs

# Shortest path Utils
def get_selected_arcs(arc_idxs, selected_arcs):
    arc = []
    for idx, i in enumerate(selected_arcs):
        if np.isclose(i, 1, rtol=1e-05, atol=1e-08, equal_nan=False): # Vecinity
            arc.append(arc_idxs[idx])
    return arc

def get_usage_string(arc_idxs, res_flow, capacity):
    return {arc: '%s/%s' % (flow, cap) for arc, flow, cap in zip(arc_idxs, res_flow, capacity)}

def get_min_cut(arc_idxs, np_res_flow, np_capacity):
    np_capacity = np.where(np_capacity == None, 999, np_capacity)

In [23]:
# IMPORT THE DATA:
NN = np.array([
        [0, 0, 0, 1, 1],
        [0, 0, 0, 1, 1],
        [0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]
    ])

beq = np.array([10, 20, 15, -25, -20])
C = np.array([10, 20, 10, 10, 10, 30])
Aeq, arc_idxs = nn2na(NN)
bounds = tuple([(0, None) for i in range(0, Aeq.shape[1])])

print('## Optimizer inputs ## \n'
      'Cost vector: %s \n '
      'A_eq Node-Arc matrix:\n%s \n'
      'b_eq demand-supply vector: %s \n'
      'Bounds of each X arc variable: %s \n' % (C, Aeq, beq, bounds))

# OPTIMIZE: 
name_method='simplex'
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method=name_method)

# GET THE SOLUTION:
selarcs = get_selected_arcs(arc_idxs, res.x)
print('## Results ##')

indexes = np.where(np.array(res.x) > 0.9)
print('Min Cost: %s' % (res.fun))

for i in indexes:
    print('%s: %s' % (arcs[i], res.x[i]))



## Optimizer inputs ## 
Cost vector: [10 20 10 10 10 30] 
 A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0]
 [ 0  0  1  1  0  0]
 [ 0  0  0  0  1  1]
 [-1  0 -1  0 -1  0]
 [ 0 -1  0 -1  0 -1]] 
b_eq demand-supply vector: [ 10  20  15 -25 -20] 
Bounds of each X arc variable: ((0, None), (0, None), (0, None), (0, None), (0, None), (0, None)) 

## Results ##
Min Cost: 450.0
['1A' '2B' '3A']: [10. 20. 15.]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
